In [1]:
import os, csv, json
from dotenv import load_dotenv
import time
from datetime import datetime
import schedule
import requests

import pandas as pd
import pandas_ta as ta
import numpy as np

import yfinance as yf
yf.pdr_override()
from google.oauth2 import service_account
import pygsheets

load_dotenv()

True

In [2]:
with open("..\service_account.json") as source:
    info = json.load(source)
credentials = service_account.Credentials.from_service_account_info(info)

CLIENT = pygsheets.authorize(service_account_file='..\service_account.json')
SHEET_URL = os.getenv('TRADE_SHEET')
data = CLIENT.sheet.get(os.getenv('TRADE_SHEET_ID'))
data['spreadsheetId']

'1WTJj8qkV9GeM_Fz8m2bLH3gZNNpSfRnqtzBiv3XaFsc'

In [3]:
LINE_TOKEN = os.getenv('LINE_TOKEN')
LINE_URL = 'https://notify-api.line.me/api/notify'
LINE_HEADERS = {'content-type':'application/x-www-form-urlencoded','Authorization':'Bearer ' + LINE_TOKEN }

In [4]:
def writeTradeLog(row, pair): # only xStation log only
    new_row = [] # convert data from Xstation.
    sheet = CLIENT.open_by_url(SHEET_URL)
    worksheet = sheet.worksheet_by_title('FowardTest')
    cells = worksheet.get_all_values(include_tailing_empty_rows=False, include_tailing_empty=False, returnas='matrix')
    last_row = len(cells)

    new_row.append([
        str(last_row), pair, row['action'], row['open time'], row['open'], row['close time'], row['close'],
        row['result'], row['strategy'], row['size']
    ])
    
    worksheet = worksheet.insert_rows(last_row, number=1, values= new_row)
    print(last_row, new_row)
    print('done...')

In [5]:
def notifyMsg(*msg):
    r = requests.post(LINE_URL, headers=LINE_HEADERS, data={'message': msg[0]})
    print(r.text)

In [10]:
class WilliamsRNotify:
    def __init__(self, params):
        self.name = 'Williams%R'
        
        self.willr_status = 'idle' # idle, overbought, oversold
        self.pair = params['pair']
        self.tf = params['tf']
        self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                      'close': None, 'T/P': None, 'S/L': None,'result': None, 'strategy': self.name, 'size': 0}
        
        self.current_action = 'close'
        self.atr = params['atr']
        self.ema = params['ema']
        self.willr = params['willr']
        
        self.overbought = params['overbought']
        self.oversold = params['oversold']
        
        self.pip = params['pip']
        self.pip_value = params['pip_value']
        self.rr = params['rr']
        
        self.lot_size = 0.01
        self.target = 10
        self.round = params['round']
        self.df = None
        
    def get_bars(self, period_bars='3d'):
        ticker_yahoo = yf.Ticker(self.pair+'=x')
        bars = ticker_yahoo.history(period=period_bars, interval=self.tf)
        df = pd.DataFrame({
            'datetime': bars.index,
            'open': bars['Open'],
            'high': bars['High'],
            'low': bars['Low'],
            'close': bars['Close'],
            'volumn': bars['Volume']
        }) # convert alpha vantage data to use with strategy

        df['timestamp'] = pd.to_datetime(df['datetime']).values.astype(np.int64) // 10**9
        df['datetime'] = pd.to_datetime(df['timestamp'] * 10**9, utc=True).dt.tz_convert('Asia/Bangkok')
        df.reset_index(drop=True, inplace=True)
        return df
        
    def initial_bars(self):
        init_df = self.get_bars(period_bars='3d')
        self.df = init_df
        
    def reset_order(self):
        self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                      'close': None, 'T/P': None, 'S/L': None,'result': None, 'strategy': self.name, 'size': 0}
    
    def calLotSize(self, entry, exit):
        self.lot_size = abs(round(self.target / (((entry - exit) / self.pip ) * self.pip_value ) , 2))
        print(f'lot_size: {str(self.lot_size)}')
        
    def longCondition(self, row):
        if self.willr_status == 'idle':
            if (row['EMA_'+str(self.ema)] < row['close']) and (row['WILLR_'+str(self.willr)] < self.oversold):
                self.willr_status = 'oversold'
                        
            elif (row['EMA_'+str(self.ema)] > row['close']) and (row['WILLR_'+str(self.willr)] > self.overbought):
                self.willr_status = 'overbought'
                        
        elif self.willr_status == 'oversold':
            if (row['EMA_'+str(self.ema)] < row['close']) and (row['WILLR_'+str(self.willr)] > -50):
                row['entries'] = True
                self.current_action = 'long'
                
                self.latest_sl = row['open'] - row['ATRr_'+str(self.atr)]
                self.latest_tp = row['open'] + (self.rr * abs(row['open'] - self.latest_sl))
                row['lot_size'] = self.calLotSize(row['open'], self.latest_tp)        
                self.willr_status = 'idle'
                        
            elif (row['EMA_'+str(self.ema)] > row['close']):
                self.willr_status = 'idle'
        
        if (row['EMA_'+str(self.ema_1)] > row['EMA_'+str(self.ema_2)]) and (row['low'] > row['EMA_'+str(200)]):
            # check if previous n bars ema_1 < ema_2 then buy.
            if row['prev_ema_1'] < row['prev_ema_2']:
                row['action'] = 'long'
                self.order['action'] = 'long'
                self.order['open time'] = row['timestamp']
                self.order['open'] = row['open']
                self.order['S/L'] = row['open'] - row['ATRr_'+str(self.atr_len)]
                self.order['T/P'] = row['open'] + (self.rr * abs(row['open'] - self.order['S/L']))
                self.current_action = 'buy'
                        
                notifyMsg(f"Buy {self.pair} {self.name}\nprice: {str(self.order['open'])}\nTP: {str(self.order['T/P'])}\nSL: {str(self.order['S/L'])}")
                self.calLotSize(self.order['open'], self.order['T/P'])
                
    def longResult(self, row):
        if (row['low'] <= self.order['S/L']):
            self.order['result'] = 'S/L'
            self.order['close time'] = row['timestamp']
            self.order['close'] = self.order['S/L']
                
            self.current_action = 'close'
            self.notifyMsg(f"S/L Buy {self.pair} {self.name}\nprice: {self.order['close']}")
            self.order['size'] = self.lot_size
            writeTradeLog(self.order, self.pair)
                
            self.reset_order()
                
        elif (row['high'] >= self.order['T/P']):
            self.order['result'] = 'T/P'
            self.order['close time'] = row['timestamp']
            self.order['close'] = self.order['T/P']
                
            self.current_action = 'close'
            self.notifyMsg(f"T/P Buy {self.pair} {self.name}\nprice: {self.order['close']}")
            self.order['size'] = self.lot_size
            writeTradeLog(self.order, self.pair)
                
            self.reset_order()
        
    def shortCondition(self, row):
        if (row['EMA_'+str(self.ema_1)] < row['EMA_'+str(self.ema_2)]) and (row['high'] < row['EMA_'+str(200)]):
            # check if previous n bars ema_1 > ema_2 then sell.
            if row['prev_ema_1'] > row['prev_ema_2']:
                row['action'] = 'short'
                self.order['action'] = 'short'
                self.order['open time'] = row['timestamp']
                self.order['open'] = row['open']
                self.order['S/L'] = row['open'] + row['ATRr_'+str(self.atr_len)]
                self.order['T/P'] = row['open'] - (self.rr * abs(row['open'] - self.order['S/L']))
                self.current_action = 'sell'
                        
                notifyMsg(f"Sell {self.pair} {self.name}\nprice: {str(self.order['open'])}\nTP: {str(self.order['T/P'])}\nSL: {str(self.order['S/L'])}")
                self.calLotSize(self.order['open'], self.order['T/P'])
                
    def shortResult(self, row):
        if (row['high'] >= self.order['S/L']):
            self.order['result'] = 'S/L'
            self.order['close time'] = row['timestamp']
            self.order['close'] = self.order['S/L']
                
            self.current_action = 'close'
            self.notifyMsg(f"S/L Sell {self.pair} {self.name}\nprice: {self.order['close']}")
            self.order['size'] = self.lot_size
            writeTradeLog(self.order, self.pair)
                
            self.reset_order()
                
        elif (row['low'] <= self.order['T/P']):
            self.order['result'] = 'T/P'
            self.order['close time'] = row['timestamp']
            self.order['close'] = self.order['T/P']
                
            self.current_action = 'close'
            self.notifyMsg(f"T/P Sell {self.pair} {self.name}\nprice: {self.order['close']}")
            self.order['size'] = self.lot_size
            writeTradeLog(self.order, self.pair)
                
            self.reset_order()
        
    def createStrategy(self):
        gloden_cross = ta.Strategy(
            name = 'Gloden cross',
            description = 'Gloden cross for alert',
            ta = [
                {'kind': 'ema', 'length': self.ema_1},
                {'kind': 'ema', 'length': self.ema_2},
                {'kind': 'ema', 'length': 200},
                {'kind': 'atr', 'length': self.atr}
            ]
        )
        self.df.ta.strategy(gloden_cross)
        self.df['prev_ema_1'] = self.df['EMA_'+str(self.ema_1)].shift(1)
        self.df['prev_ema_2'] = self.df['EMA_'+str(self.ema_2)].shift(1)
        
    def realtime_long(self):
        incoming_df = self.get_bars(period_bars='1d')
        if (incoming_df['timestamp'].iloc[0] != self.df['timestamp'].iloc[-1]):
            self.df = self.df.append(incoming_df.tail(1), ignore_index = True)
        self.createStrategy()
        
        row = self.df.tail(1).to_dict('records')[0]
        if self.current_action == 'close':
            self.longCondition(row)
        else:
            self.longResult(row)
    
    def realtime_short(self):
        incoming_df = self.get_bars(period_bars='1d')
        if (incoming_df['timestamp'].iloc[0] != self.df['timestamp'].iloc[-1]):
            self.df = self.df.append(incoming_df.tail(1), ignore_index = True)
        self.createStrategy()
        
        row = self.df.tail(1).to_dict('records')[0]
        if self.current_action == 'close':
            self.shortCondition(row)
        else:
            self.shortResult(row)
    
    def realtime_both(self):
        incoming_df = self.get_bars(period_bars='1d')
        if (incoming_df['timestamp'].iloc[0] != self.df['timestamp'].iloc[-1]):
            self.df = self.df.append(incoming_df.tail(1), ignore_index = True)
        self.createStrategy()
        
        row = self.df.tail(1).to_dict('records')[0]
        if self.current_action == 'close':
            self.longCondition(row)
            self.shortCondition(row)
            
        elif self.current_action == 'long':
            self.longResult(row)
                    
        elif self.current_action == 'short':
            self.shortResult(row)
    

In [11]:
audusd = WilliamsRNotify({
    'pair': 'AUDUSD',
    'tf': '15m',
    'atr': 14,
    'ema': 100,
    'willr': 20,
    'overbought': -20,
    'oversold': -80,
    'pip': 0.0001,
    'pip_value': 10,
    'rr': 1.5,
    'round': 5
})

In [12]:
audusd.initial_bars()

In [13]:
def get_all_realtime():
#     audusd.get_realtime()
    audusd.realtime_both()
    
schedule.every(60 * 15).seconds.do(get_all_realtime) 

Every 900 seconds do get_all_realtime() (last run: [never], next run: 2022-08-11 17:03:41)

In [14]:
while True:
    schedule.run_pending()
    time.sleep(1)

KeyboardInterrupt: 